# Subgraph Extraction
# Does not run on Pascal

In this notebook, we will use cuGraph to extract a subgraph from the test graph.  

| Author Credit |    Date    |  Update          | cuGraph Version |  Test Hardware              |
| --------------|------------|------------------|-----------------|-----------------------------|
| Brad Rees     | 10/16/2019 | created          | 0.13            | GV100 32G, CUDA 10.2        |
|               | 08/16/2020 | updated          | 0.15   | GV100 32G, CUDA 10.2        |
| Don Acosta    | 07/11/2022 | tested / updated | 22.08 nightly   | DGX Tesla V100 CUDA 11.5    |
| Ralph Liu    | 07/26/2022 | updated | 22.08 nightly   | DGX Tesla V100 CUDA 11.5    |

## Introduction

Subgraph Extraction allows the extraction of a portion of a graph and returned as a Graph object that follow-on analytics can be applied against.  For example, many of the community detection analytics simply return a list of vertices in a cluster.  In order to run follow-on processing against a cluster, all the edges within that cluster are also needed.  

For this notebook, we will run Louvain to determine the clusters and then extract one of the clusters using Subgraph Extraction.  We will then explore that extracted subgraph.  



To compute the Louvain cluster in cuGraph use: <br>
* __G', df = cugraph.subgraph(G, v)__
    * G: A cugraph.Graph object
    * v: cudf.Series - list of vertex IDs to be extracted

Returns:<br>
* G': A cugraph.Graph object
* df: cudf.DataFrame
    * old - the original vertex ID
    * new - the new renumbered vertex ID



#### Some notes about vertex IDs...

* cuGraph will automatically renumber graphs to an internal format consisting of a contiguous series of integers starting from 0, and convert back to the original IDs when returning data to the caller. If the vertex IDs of the data are already a contiguous series of integers starting from 0, the auto-renumbering step can be skipped for faster graph creation times.
  * To skip auto-renumbering, set the `renumber` boolean arg to `False` when calling the appropriate graph creation API (eg. `G.from_cudf_edgelist(gdf_r, source='src', destination='dst', renumber=False)`).
  * For more advanced renumbering support, see the examples in `structure/renumber.ipynb` and `structure/renumber-2.ipynb`


### Test Data
We will be using the Zachary Karate club dataset 
*W. W. Zachary, An information flow model for conflict and fission in small groups, Journal of
Anthropological Research 33, 452-473 (1977).*

<img src="../../img/zachary_graph_clusters.png" width="35%"/>


### Prep

In [ ]:
# Import needed libraries
import cugraph
import cudf

# Import a built-in dataset
from cugraph.experimental.datasets import karate

## Read data using cuDF

#### Run Louvain to find a cluster to extract
Since the focus is on the subgraph extraction portion, we are going to leave the vertex IDs are they are and just accept that there is an extract cluster in the reults with vertex 0

In [ ]:
# You can also just get the edgelist
gdf = karate.get_edgelist(fetch=True)

# The louvain algorithm requires that there are vertex weights.  Just use 1.0 
gdf["data"] = 1.0

# Create a graph
G = cugraph.from_cudf_edgelist(gdf, source='src', destination='dst', edge_attr='data')

In [ ]:
# Call Louvain on the graph
df, mod = cugraph.louvain(G) 

In [ ]:
# How many partitions where found
part_ids = df["partition"].unique()

In [ ]:
print(str(len(part_ids)) + " partition detected")

In [ ]:
for p in range(len(part_ids)):
    part = []
    for i in range(len(df)):
        if (df['partition'].iloc[i] == p):
            part.append(df['vertex'].iloc[i])
    print("Partition " + str(p) + ":")
    print(part)


### Run Subgraph Extraction
Let's extract cluster 1

In [ ]:
vids = df.query("partition == 1")

In [ ]:
v = cudf.Series(vids['vertex'])
len(v)

In [ ]:
subG = cugraph.subgraph(G, v)

In [ ]:
print("Main Graph")
print("\tNumber of Vertices: " + str(G.number_of_vertices()))
print("\tNumber of Edges:    " + str(G.number_of_edges()))

In [ ]:
print("Subgraph")
print("\tNumber of Vertices: " + str(subG.number_of_vertices()))
print("\tNumber of Edges:    " + str(subG.number_of_edges()))

In [ ]:
coo = cudf.DataFrame()
subDF = subG.view_edge_list()

In [ ]:
len(subDF)

___
Copyright (c) 2019-2022, NVIDIA CORPORATION.

Licensed under the Apache License, Version 2.0 (the "License");  you may not use this file except in compliance with the License. You may obtain a copy of the License at http://www.apache.org/licenses/LICENSE-2.0

Unless required by applicable law or agreed to in writing, software distributed under the License is distributed on an "AS IS" BASIS, WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied. See the License for the specific language governing permissions and limitations under the License.
___